In [1]:
# import libs

import numpy as np
import Laser
import gCode
import random
import Printer


# Important definition
positive angle in CCW direction

### Units
Distances: mm  
Angles: degrees

# Define functions

In [2]:
# read G-Code file and return a list of strings
def read_gcode_file(filename):
    tmp = []
    with open(filename, 'r') as f:
        gcode = f.readlines()
    for line in gcode:
        tmp.append(line)
    
    return np.array(tmp)
    

In [3]:
def parseCoord(gCodeObj,linestring):
    line = linestring
    X = None
    Y = None
    Z = None
    E = None
    F = None
    move = False

    # check the command of the gcode line (string until first space)
    command = line.split(' ')[0]

    # check if line is ;LAYER:1 and gCodeObj.firstLayerCheck is False
    if line.startswith(';LAYER:1') and not gCodeObj.firstLayerCheck:
        gCodeObj.updateFirstLayerCheck(True)
        print("First layer check is True")

    # check if firstLayerCheck is True
    if gCodeObj.firstLayerCheck:
        # if command == G1 or G0 than move = True
        if command == 'G1' or command == 'G0':
            move = True
        if command == 'G1':
            gCodeObj.updateNumberOfG1()
        # check if line contains ; which means it is a comment and skip line
        if not ';' in line:
            # parse the X Y Z coords from string to float
            if 'X' in line:
                X = float(line.split('X')[-1].split(' ')[0])
            elif 'X' not in line:
                X = None
            if 'Y' in line:
                Y = float(line.split('Y')[-1].split(' ')[0])
            elif 'Y' not in line:
                Y = None
            if 'Z' in line:
                Z = float(line.split('Z')[-1].split(' ')[0])
            elif 'Z' not in line:
                Z = None
            if 'E' in line:
                E = float(line.split('E')[-1].split(' ')[0])
            elif 'E' not in line:
                E = None
            if 'F' in line:
                F = float(line.split('F')[-1].split(' ')[0])
                gCodeObj.updateLastF(F)
            elif 'F' not in line:
                F = gCodeObj.getLastF()
    else:
        X = None
        Y = None
        Z = None
        E = None
        F = None

    return move, X, Y, Z, E, F, command, linestring


In [4]:
# calculate the distance between two points
def calcDistance(x0, y0, x1, y1):
    return np.sqrt((x1-x0)**2 + (y1-y0)**2)


In [5]:
# calculate angle between two points (atan2)
def calcAngle(x0, y0, x1, y1):
    newAngle = 0
    try:
        newAngle = np.arctan2((y1-y0),(x1-x0))
   
    except:
        print("Error in calcAngle", x0, y0, x1, y1)
    # convert to degrees
    newAngle = np.rad2deg(newAngle)
    
    # normalize angle
    #newAngle = np.mod(newAngle, 360)
    
    return newAngle

In [6]:
def findClosestAngle(targetAngle, LaserObj):
    currentAngles = LaserObj.getNormAngles()
    
    # find the closest angle in the list of angles
    closestAngleIndx = np.argmin(np.abs(currentAngles - targetAngle))
    closestAngle = currentAngles[closestAngleIndx]

    # calculte delta bewteen target and closest angle
    deltaAngle = targetAngle - closestAngle

    # update the angle of the laser
    #LaserObj.updateAngles(deltaAngle)

    return closestAngleIndx, deltaAngle

    

In [7]:
def convertAngleToSteps(angle):
    return angle


In [8]:
def checkNextLaser(pos0, pos1, LaserObj):
    X0 = pos0[0]
    Y0 = pos0[1]

    X1 = pos1[0]
    Y1 = pos1[1]

    # calculate the angle between the two points
    targetAngle = calcAngle(X0, Y0, X1, Y1)  

    # noralize the target angle
    targetAngle = np.mod(targetAngle, 360)

    # find the closest angle to the target angle
    closestAngleIndx, deltaAngle = findClosestAngle(targetAngle, LaserObj)

    return closestAngleIndx


In [14]:
def postProcessGcode(gCodeArray, threshold, LaserObj, newFilePath, gCodeObj):

    for i in range(gCodeArray.shape[0]-2):              # Die letzte Zeile wird nicht mehr betrachtet
        move0, X0, Y0, Z0, E0, F0, command0, line0 = parseCoord(gCodeObj, gCodeArray[i])
        move1, X1, Y1, Z1, E1, F1, command1, line1 = parseCoord(gCodeObj, gCodeArray[i+1])
        move2, X2, Y2, Z2, E2, F2, command2, line2 = parseCoord(gCodeObj, gCodeArray[i+2])


        # update lastPos if move1 is False and move0 is True
        if move0 and not move1:
            gCodeObj.updateCurrentPos([X0, Y0, Z0])

        # clear Extruder for testing
        if(0):
            # copy line0 until reaches E 
            line0 = line0.split('E')[0] + '\n'

        # check if both lines are moves
        if move0 and move1:
            # check if the distance between the two points is larger than the threshold
        
            #print("i:", i+1, "move0: ", move0, "move1: ", move1, "X0: ", X0, "Y0: ", Y0, "Z0: ", Z0, "command0: ", command0, "line0: ", line0, "X1: ", X1, "Y1: ", Y1, "Z1: ", Z1, "command1: ", command1, "line1: ", line1)
            dist = calcDistance(X0, Y0, X1, Y1)
            gCodeObj.updateTotalLengthOfG1(dist)

            # calculate the angle between the two points
            targetAngle = calcAngle(X0, Y0, X1, Y1)
            targetAngle2 = calcAngle(X1, Y1, X2, Y2)

            # update the last angle
            gCodeObj.updateLastAngle(targetAngle)

            # war vorher in der if schleife
            # noralize the target angle
            targetAngle = np.mod(targetAngle, 360)

            # find the closest angle to the target angle
            closestAngleIndx, deltaAngle = findClosestAngle(targetAngle, LaserObj)
            closestAngleIndx2, deltaAngle2 = findClosestAngle(targetAngle2, LaserObj)

            # update the angle of the laser
            #LaserObj.updateAngles(deltaAngle)

            # update absolute total laser movement
            LaserObj.updateAbsLaserMovement(deltaAngle)
            
            # corrected laser connection
            laserIndx = closestAngleIndx + 2

            # string for commands from if-statements
            startLine = ""
            endLine = ""
            commandLine = ""

            
            absAngle = LaserObj.getAbsLaserMovement()
            # move X Y pos to next point and extrude filament
            commandLine += str(command1) + " F"+ str(F1) +" X" + str(X1) + " Y" + str(Y1) + " E" + str(E1) + " A" + str(absAngle) +" ;" + str(laserIndx) + "\n"

            # if distance is larger than threshold -> lasers are used
            if (dist > threshold):
                # activate laser
                startLine += "M106 P" + str(laserIndx) + " S1 \n"

                # deactivate laser
                endLine = "M106 P" + str(laserIndx) + " S0 \n"

            # check if the laser is already activated
            if (closestAngleIndx != LaserObj.getLastActivatedLaser()):
                # deactivate last laser
                startLine += "M106 P" + str(LaserObj.getLastActivatedLaser()) + " S0 \n"

                # activate laser
                startLine += "M106 P" + str(laserIndx) + " S1 \n"

                # deactivate laser
                endLine = "M106 P" + str(laserIndx) + " S0 \n"

            

            # combine the lines
            Line = startLine + commandLine + endLine

            # write the line to the new file
            with open(newFilePath, 'a') as f:
                f.write(Line)
                f.close()

            # save last used laser
            LaserObj.updateLastActivatedLaser(closestAngleIndx)
                         
        else:
            # append line0 to newFilePath
            with open(newFilePath, 'a') as f:
                f.write(line1)
                f.close()

    # turn off all lasers
    for i in range(LaserObj.getNumberOfLasers()):
        Line = "M106 P" + str(i+2) + " S0 \n"
        with open(newFilePath, 'a') as f:
            f.write(Line)
            f.close()
            
        

    return True

# Test functions

In [10]:
# test calcAngle function
if (0):
    print(calcAngle(0,0,0,0))
    print(calcAngle(0,0,1,-1))
    print(calcAngle(0,0,-1,-1))
    print(calcAngle(0,0,-1,1))


In [11]:
# test the parseCoord function
if(0):

# read the gcode file
    gcode = read_gcode_file("GCODE.txt")

    # create empty lists for the coordinates
    X = []
    Y = []
    Z = []
    cmd = []
    i = 0
    # loop over all lines in the gcode file
    for line in gcode:
        #print(i)
        #i += 1
        # parse the coordinates from the line
        moveBool, x, y, z, command, linestring = parseCoord(line)
        # append the coordinates to the lists
        X.append(x)
        Y.append(y)
        Z.append(z)
        cmd.append(command)

# print first 10 elements of X Y Z
j = 50
if(0):
    for i in range(j):
        print(f"X: {X[i]} Y: {Y[i]} Z: {Z[i]} cmd: {cmd[i]}")

# Main 

In [12]:
# initiate Printer object, for duration calculation of process
Prusa = Printer.Printer(velocity_trans=100, velocity_angular=100)

# read the gcode file
gcode = read_gcode_file("Input/PI3MK3M_Square_50mm.gcode")
gCodeObj = gCode.gCode("gCode")

# print shape of gcode
print(gcode.shape)

# create laser object
numberOfLaser = 4
laser = Laser.Laser(numberOfLaser)

# threshold for the distance between two points 
minDistance = 2

# check index in Output/index.txt
with open("Output/index.txt", 'r') as f:
    index = int(f.read())
    f.close()

# create new file
newFilePath = "Output/GCODE_postprocessed_" + str(index) + ".gcode"

# iterate index in Output/index.txt
with open("Output/index.txt", 'w') as f:
    f.write(str(index+1))
    f.close()

# post process the gcode
print(postProcessGcode(gcode, minDistance, laser, newFilePath, gCodeObj))


(120,)
First layer check is True
Error in calcAngle 149.8 129.8 None None
Error in calcAngle 149.02 129.02 None None
Error in calcAngle 138.749 128.82 None None
Error in calcAngle 149.4 129.4 None None
Error in calcAngle 149.8 129.8 None None
Error in calcAngle 149.02 129.02 None None
True


In [13]:
print(gCodeObj.firstLayerCheck)
print(laser.absLaserMovement)
print("Total lenght of G1 commands:\t " + str(gCodeObj.getTotalLengthOfG1()) + " mm")
print("Total angle turned: \t" + str(laser.getAbsLaserMovement()))
print("Total number of G1 commands: \t\t" + str(gCodeObj.getNumberOfG1()))



True
45.0
Total lenght of G1 commands:	 1283.2232335787926 mm
Total angle turned: 	45.0
Total number of G1 commands: 		179
